## Object Tracking Using Color Detection (Pankaj Pundir)

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
def im(image,name="image"):
    cv2.imshow(name,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
lines=[]
def center_of_contour(x_c):
    M=cv2.moments(x_c)
    #print(M['m00'])
    return (int(M['m10']/(M['m00'])),int(M['m01']/(M['m00'])))
def x_cord_contour(x_c):
    if cv2.contourArea(x_c) >9:
        return center_of_contour(x_c)[0]
    
def image_color_track(black,image):
    gray=cv2.cvtColor(black,cv2.COLOR_RGB2GRAY)
    can=cv2.Canny(gray,3,20)
    con_img,contours,hierarchy=cv2.findContours(can,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #print("success")
    try:
        #sorted_cont=sorted(contours,key=cv2.contourArea,reverse=True)
        area,ci=0,0
        for i in range(len(contours)):
            cnt=contours[i]
            max_area = cv2.contourArea(cnt)
            if(area>max_area):
                max_area=area
                ci=i
        cnt=contours[ci]
        hull=cv2.convexHull(cnt)
        cv2.drawContours(black,[hull],0,(0,255,0),2)
        #print("success")
        try:
            lines.append((center_of_contour(cnt)+lines[-1])/2)
        except:
            lines.append((center_of_contour(cnt)))
    finally:
        if len(lines)>50:
            lines.pop(0)
        if len(lines)>0:
            for i in range(len(lines)-1):
                cv2.line(image,lines[i],lines[i+1],(255,255,255),3)
        return image
def color_extractor(image): 
    #hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    hsv=image
    lower_lim=np.array([0,0,100],dtype='uint8')
    upper_lim=np.array([0,0,255],dtype='uint8')
    mask=cv2.inRange(hsv,lower_lim,upper_lim)
    res=cv2.bitwise_and(hsv,hsv,mask=mask)
    blur = cv2.GaussianBlur(res,(5,5),0)
    kernel=np.ones((10,10),np.uint8)
    res=cv2.dilate(res,kernel,iterations=1)
    #res=cv2.blur(res,(8,8))
    #return res
    return image_color_track(res,image)

In [7]:
cap = cv2.VideoCapture(0)
# VideoCapture cap
# cap.open(192.168.1.180/?action=stream?dummy=param.mjpg)
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))




if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
 
    # Display the resulting frame
    cv2.imshow('Frame',color_extractor(frame))
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()